In [41]:
import pandas as pd
import numpy as np
import os
import matplotlib.pyplot as plt

from prophet import Prophet


In [42]:
data_folder = "data"
filename = "train.csv"
df = pd.read_csv(os.path.join("..", data_folder, filename))

df_subset = df[["warehouse", "orders", "date"]]

df_subset.rename(columns={"date": "ds",
                              "orders": "y"}, inplace=True)

/tmp/ipykernel_9852/3729658863.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_subset.rename(columns={"date": "ds",


In [43]:
warehouses = df_subset["warehouse"].unique().tolist()

In [44]:
filename1 = "test.csv"
df_test = pd.read_csv(os.path.join("..", data_folder, filename1))

df_test_subset = df_test[["date", "warehouse"]].rename(columns={"date": "ds"})

In [49]:
def hyper_tuning(df, param_grid):
    from sklearn.model_selection import ParameterGrid

    # Create a list of all combinations of hyperparameters
    all_params = list(ParameterGrid(param_grid))


    # Cross-validation function to calculate MAPE
    def cross_validation_mape(model, initial, period, horizon):
        from prophet.diagnostics import cross_validation
        from prophet.diagnostics import performance_metrics

        df_cv = cross_validation(model, initial=initial, period=period, horizon=horizon)
        df_p = performance_metrics(df_cv, metrics=['mape'])
        return df_p['mape'].values[0]

    list_of_result_dicts = []
    list_of_scores = []
    # Perform grid search
    for params in all_params:

        m = Prophet(
            changepoint_prior_scale=params['changepoint_prior_scale'],
            seasonality_prior_scale=params['seasonality_prior_scale'],
            holidays_prior_scale=params['holidays_prior_scale'],
            seasonality_mode=params['seasonality_mode']
        )
        m.fit(df)

        # Define the initial, period, and horizon for cross-validation
        initial = '365 days'
        period = '180 days'
        horizon = '365 days'

        # Compute MAPE using cross-validation
        mape = cross_validation_mape(m, initial, period, horizon)

        # Store the results
        list_of_result_dicts.append(params)
        list_of_scores.append(mape)

    results = pd.DataFrame({"params": list_of_result_dicts,
                            "MAPE": list_of_scores})

    # Find the best hyperparameters
    best_params = results.loc[results['MAPE'].idxmin()]

    return best_params


In [50]:
all_predictions = []

for warehouse in warehouses:

    current_df  = df_subset[df_subset["warehouse"] == warehouse].drop(columns=["warehouse"])
    print(current_df.head())
    current_df_test = df_test_subset[df_test_subset["warehouse"] == warehouse].drop(columns=["warehouse"])

    # optimal_params = {'changepoint_prior_scale': 0.8,
    #               'holidays_prior_scale': 0.001,
    #               'seasonality_mode': 'additive',
    #               'seasonality_prior_scale': 0.02}

    # best_munich_params = {'changepoint_prior_scale': 0.2,
    #                   'holidays_prior_scale': 0.001,
    #                   'seasonality_mode': 'multiplicative',
    #                   'seasonality_prior_scale': 0.4}

    param_grid = {
    'changepoint_prior_scale': [0.2, 0.3, 0.4, 0.8],
    'seasonality_prior_scale': [0.02, 0.2, 0.4],
    'holidays_prior_scale': [0.001, 0.002],
    'seasonality_mode': ['additive', 'multiplicative']
    }

    optimal_params = hyper_tuning(current_df, param_grid)

    model = Prophet(**optimal_params["params"])

    model.fit(current_df)


    forecast = model.predict(current_df_test)

    all_predictions += list(forecast["yhat"])

print(all_predictions)

        y          ds
0  6895.0  2020-12-05
1  6584.0  2020-12-06
2  7030.0  2020-12-07
3  6550.0  2020-12-08
4  6910.0  2020-12-09


19:37:15 - cmdstanpy - INFO - Chain [1] start processing
19:37:15 - cmdstanpy - INFO - Chain [1] done processing
Seasonality has period of 365.25 days which is larger than initial window. Consider increasing initial.


  0%|          | 0/3 [00:00<?, ?it/s]

19:37:15 - cmdstanpy - INFO - Chain [1] start processing
19:37:15 - cmdstanpy - INFO - Chain [1] done processing
19:37:15 - cmdstanpy - INFO - Chain [1] start processing
19:37:15 - cmdstanpy - INFO - Chain [1] done processing
19:37:16 - cmdstanpy - INFO - Chain [1] start processing
19:37:16 - cmdstanpy - INFO - Chain [1] done processing
19:37:16 - cmdstanpy - INFO - Chain [1] start processing
19:37:16 - cmdstanpy - INFO - Chain [1] done processing
Seasonality has period of 365.25 days which is larger than initial window. Consider increasing initial.


  0%|          | 0/3 [00:00<?, ?it/s]

...

  0%|          | 0/3 [00:00<?, ?it/s]

19:45:52 - cmdstanpy - INFO - Chain [1] start processing
19:45:52 - cmdstanpy - INFO - Chain [1] done processing
19:45:52 - cmdstanpy - INFO - Chain [1] start processing
19:45:52 - cmdstanpy - INFO - Chain [1] done processing
19:45:52 - cmdstanpy - INFO - Chain [1] start processing
19:45:53 - cmdstanpy - INFO - Chain [1] done processing
19:45:53 - cmdstanpy - INFO - Chain [1] start processing


Best parameters: params    {'changepoint_prior_scale': 0.8, 'holidays_pri...
MAPE                                               0.051022
Name: 38, dtype: object


19:45:53 - cmdstanpy - INFO - Chain [1] done processing


[9811.29013037464, 9780.438280716055, 10032.83639190523, 9752.995986578237, 9765.478297930267, 9969.997548725292, 10812.3288647689, 9769.681963996616, 9754.621928936986, 10022.66083228047, 9757.704748633001, 9783.741824424384, 9999.964792113895, 10851.696258836711, 9815.77736985887, 9804.500692843696, 10073.210723716962, 9805.756282007547, 9826.180313692024, 10033.84044941579, 10874.328725982694, 9824.851262651186, 9798.14903611864, 10050.083550196727, 9765.07214786249, 9767.759033041199, 9958.115951474125, 10782.341573712261, 9718.221020403284, 9679.02731608024, 9921.089183375605, 9629.210495872152, 9628.327964365151, 9818.601434670778, 10646.306176718215, 9589.188839474797, 9560.368982573653, 9815.91060769264, 9540.248905005794, 9557.858575693348, 9768.357790376715, 10617.416750143653, 9582.13202938707, 9574.948048959264, 9851.249490144019, 9594.809221111112, 9629.477441409905, 9854.305457456663, 10714.472436964501, 9686.675750112945, 9683.065739346372, 9958.848302204928, 9697.739526

In [51]:
final_result = pd.concat([df_test["id"], pd.DataFrame({"orders": all_predictions})], axis=1)


In [54]:
final_result

,id,orders
0,Prague_1_2024-03-16,9811.290130
1,Prague_1_2024-03-17,9780.438281
2,Prague_1_2024-03-18,10032.836392
3,Prague_1_2024-03-19,9752.995987
4,Prague_1_2024-03-20,9765.478298
...,...,...
392,Budapest_1_2024-05-11,6647.729134
393,Budapest_1_2024-05-12,6317.400190
394,Budapest_1_2024-05-13,6570.870536
395,Budapest_1_2024-05-14,6627.193590


In [52]:
final_result.to_csv("submission.csv", index=False)